In [1]:
spark

In [8]:
spark.conf.set("viewsEnabled","true")
spark.conf.set("materializationDataset","<dataset>")

eth_tx = spark.read.format("bigquery")\
    .option('table', 'bigquery-public-data:crypto_ethereum.transactions') \
    .load()

eth_tx.createOrReplaceTempView('eth_tx')


In [10]:
sql_hash = """
SELECT hash FROM eth_tx WHERE DATE(block_timestamp) = "2022-01-11" LIMIT 1000
"""
tx_hash = spark.sql(sql_hash)

In [11]:
tx_hash.show()

+--------------------+
|                hash|
+--------------------+
|0x40158b06d355ebb...|
|0x7b079afed80c49c...|
|0xe8062b6cf5a340b...|
|0xa725246f76cf392...|
|0x0d70a967d8e2df7...|
|0x2da1fb9ec271b27...|
|0x79bca53278e606d...|
|0xd2eae104f467733...|
|0xef6ea04eebb1ecb...|
|0x91bc553be749428...|
|0x1655231e6f1e2b4...|
|0x18e5abad0eee779...|
|0x2b1d5c356c391ab...|
|0x94a150054ccb880...|
|0x68a8ff11cad4092...|
|0xe55849aea48cd50...|
|0x67f015fed444732...|
|0xb3e1b01c841db0c...|
|0xd78ede9e23c32e3...|
|0xb6ad4a536b70a4d...|
+--------------------+
only showing top 20 rows



In [12]:
tx_hash.printSchema()

root
 |-- hash: string (nullable = false)

